### 1.Load the cvs files
I firstly upload the cvs file to DBFS, then load them to each dataframe.

In [2]:
%python
# Load cust name
URL_cust_name = "/FileStore/tables/cust_names.csv"
df_cust_name = spark.read.format("csv").option("header", "true").option("inferSchema",  "true").load(URL_cust_name)
# display(df_cust_name.head(5))

# Load dcu loation
URL_dcu_location = "/FileStore/tables/dcu_locations.csv"
df_dcu_location = spark.read.format("csv").option("header", "true").option("inferSchema",  "true").load(URL_dcu_location)
# display(df_dcu_location.head(5))

# Load endpoint location
URL_endpoint_lat_long = "/FileStore/tables/endpoint_lat_long.csv"
df_endpoint_lat_long = spark.read.format("csv").option("header", "true").option("inferSchema",  "true").load(URL_endpoint_lat_long)
df_endpoint_lat_long.count()

# Load endpoint location
URL_trpkt = "/FileStore/tables/trpkt.csv"
df_trpkt = spark.read.format("csv").option("header", "true").option("inferSchema",  "true").load(URL_trpkt)
df_trpkt.count()


Out[ 1 ]: 302482

### 2. Filter unqualified data
Some "#VALUE!" and "#N/A" error records need to be deleted. Here I didn't export the error records since just for prototype.

In [4]:
# filter #VALUE!
df_trpkt = df_trpkt.filter(df_trpkt["payload_id"]!="#VALUE!")
df_endpoint_lat_long = df_endpoint_lat_long.filter(df_endpoint_lat_long["endpoint_id"]!="#N/A")
# here we should output the error row and persist them into files

### 3. Transfer fact table

3.1 transfer datetime to day

3.2 aggragation for number of trasmission

3.3 aggragation for number of reception

3.4 left join two tables

3.5 calc redundancy rate

3.6 get cust_id, endpoint geo information

3.7 duplicate table, get dcu geo information

In [6]:
from pyspark.sql.functions import substring
# transfter datetime to day
df_trpkt = df_trpkt.withColumn("date", substring("packet_datetime",1,10))


In [7]:
from pyspark.sql.functions import countDistinct
# number of transmission
df_number_transmission = df_trpkt.groupby('date','endpoint_id').agg(countDistinct("payload_id").alias("num_transmissions"))
# df_number_transmission.show()

# number of receptions
df_number_receptions = df_trpkt.groupby('date','endpoint_id','dcu_id').agg(countDistinct("payload_id").alias("num_receptions"))

# join two tables
df_result = df_number_receptions.join(df_number_transmission, ["date","endpoint_id"],how="left")

# redundancy rate
df_result = df_result.withColumn("redundancy_rate",df_result.num_receptions/df_result.num_transmissions)
df_result.count()

Out[ 53 ]: 13798

In [8]:
import pyspark.sql.functions as sf
from pyspark.sql.functions import col
# find cust_id by dcu_id
# df_result = df_result.withColumn("cust_id", sf.lit(582))
df_result = df_result.join(df_dcu_location,["dcu_id"],how="left").drop("longitude","latitude")

# add type
df_result = df_result.withColumn("type", sf.lit("endpoint"))

# add geo information and cust_id by endpoint_id
df_result = df_result.join(df_endpoint_lat_long,["cust_id","endpoint_id"], how="left")

# duplicate table
df_result2 = df_result.drop("type","longitude","latitude")\
            .withColumn("type", sf.lit("dcu"))\
            .join(df_dcu_location,["cust_id","dcu_id"], how="left")\
            .select("cust_id",col("dcu_id").alias("endpoint_id"),"date",col("endpoint_id").alias("dcu_id"),"num_receptions","num_transmissions","redundancy_rate","type","longitude","latitude")
df_result3 = df_result2.unionByName(df_result)
df_result3.show()

+-------+-----------+----------+------+--------------+-----------------+-------------------+----+----------+---------+
cust_id|endpoint_id| date|dcu_id|num_receptions|num_transmissions| redundancy_rate|type| longitude| latitude|
+-------+-----------+----------+------+--------------+-----------------+-------------------+----+----------+---------+
 582| 21|12/19/2018| 2507| 28| 28| 1.0| dcu|-92.716135|44.227263|
 582| 2|12/19/2018| 2767| 25| 26| 0.9615384615384616| dcu|-92.636672|44.412685|
 582| 14|12/19/2018| 244| 28| 28| 1.0| dcu| -92.5897| 44.3247|
 582| 15|12/19/2018| 70| 25| 26| 0.9615384615384616| dcu| -92.74083|44.411558|
 582| 13|12/19/2018| 2164| 26| 28| 0.9285714285714286| dcu| -92.87063|44.327427|
 582| 4|12/19/2018| 717| 26| 26| 1.0| dcu|-92.952507|44.472219|
 582| 4|12/19/2018| 1782| 27| 27| 1.0| dcu|-92.952507|44.472219|
 582| 6|12/19/2018| 1223| 14| 26| 0.5384615384615384| dcu|-92.768616|44.344272|
 582| 1|12/19/2018| 2333| 27| 27| 1.0| dcu|-92.358998|44.477088|
 582| 11|12/19/2018| 1049| 25| 25| 1.0| dcu|-93.010025|44.253617|
 582| 14|12/19/2018| 594| 26| 26| 1.0| dcu| -92.5897| 44.3247|
 582| 4|12/19/2018| 1429| 9| 28|0.32142857142857145| dcu|-92.952507|44.472219|
 582| 16|12/19/2018| 917| 28| 28| 1.0| dcu|-92.834507|44.415186|
 582| 12|12/19/2018| 2780| 29| 29| 1.0| dcu|-92.956444|44.327906|
 582| 17|12/19/2018| 1688| 16| 26| 0.6153846153846154| dcu| -92.97859|44.385684|
 582| 15|12/19/2018| 1537| 8| 28| 0.2857142857142857| dcu| -92.74083|44.411558|
 582| 14|12/19/2018| 1399| 2| 27|0.07407407407407407| dcu| -92.5897| 44.3247|
 582| 16|12/19/2018| 1186| 23| 26| 0.8846153846153846| dcu|-92.834507|44.415186|
 582| 15|12/19/2018| 1141| 20| 27| 0.7407407407407407| dcu| -92.74083|44.411558|
 582| 11|12/19/2018| 824| 24| 28| 0.8571428571428571| dcu|-93.010025|44.253617|
+-------+-----------+----------+------+--------------+-----------------+-------------------+----+----------+---------+
only showing top 20 rows

In [9]:
df_result3.count()

Out[ 55 ]: 27596

In [10]:

# persist file
df_result3.coalesce(1).write.format("com.databricks.spark.csv")\
   .mode("overwrite")\
   .option("header", "true")\
   .save("test_redundancy")

### 4. write result to mysql

In [12]:
username = ""
pwd = ""

connectionProperties = {
  "user" : Username,
  "password" : pwd,
  "driver" : "com.mysql.jdbc.Driver"}
 
  
url = "jdbc:mysql://:aclara-azure-dev-server.mysql.database.azure.com"
 
#destination database table 
table = "test_redundancy"
 
#write data from spark dataframe to database
df_result3.write.mode("append").jdbc(url, table, prop)

File "<command-1021668375276183>" , line 19 
 (/destination, database, table) 
 ^ 
 SyntaxError : invalid syntax